In [ ]:
!pip install -qq timm

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import cv2
from collections import defaultdict

from tqdm.auto import tqdm

# Torch
import timm
import torch
import torchvision
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, OneCycleLR, CosineAnnealingLR

# Image Aug
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

# Visialisation
import matplotlib.pyplot as plt
%matplotlib inline

#metrics
from sklearn.metrics import mean_squared_error


In [ ]:
#Random Seed Initialize
RANDOM_SEED = 42

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.derterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything()

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

In [ ]:
train_dir = '../input/petfinder-pawpularity-score/train'
test_dir = '../input/petfinder-pawpularity-score/test'
csv_dir = '../input/petfinder-pawpularity-score/'

In [ ]:
df_train = pd.read_csv(os.path.join(csv_dir, 'train.csv'))
df_test = pd.read_csv(os.path.join(csv_dir, 'test.csv'))

In [ ]:
def return_filepath(filename, path=train_dir):
    return os.path.join(path, f'{filename}.jpg')

In [ ]:
df_train['image_path'] = df_train['Id'].apply(lambda x: return_filepath(x))
df_test['image_path'] = df_test['Id'].apply(lambda x: return_filepath(x, test_dir))

In [ ]:
df_train.head()

In [ ]:
target = 'Pawpularity'
not_features = ['Id', 'Pawpularity', 'image_path']
list_col = list(df_train.columns)
features = [i for i in list_col if i not in not_features]


In [ ]:
params = {
    'scheduler_name': 'CosineAnnealingWarmRestarts',
    'T_0': 5,
    'min_lr': 1e-7,
    'model': 'vit_large_patch32_384',
    'pretrained': True,
    'dense_features': features,
    'inp_channels': 3,
    'img_size': 384,
    'device': device,
    'lr': 1e-5,
    'weight_decay': 1e-6,
    'batch_size': 16,
    'num_workers': 0,
    'epochs':1,
    'out_features': 1,
    'dropout': 0.2
}

In [ ]:
def get_train_transforms(DIM = params['img_size']):
    return albumentations.Compose(
        [
            albumentations.Resize(DIM,DIM),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=180, p=0.7),
            albumentations.ShiftScaleRotate(
                shift_limit = 0.1, scale_limit=0.1, rotate_limit=45, p=0.5
            ),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, sat_shift_limit=0.2,
                val_shift_limit=0.2, p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1, 0.1),
                contrast_limit=(-0.1, 0.1), p=0.5
            ),
            ToTensorV2(p=1.0),
        ]
    )


In [ ]:
class PetDataset(Dataset):
    def __init__(self, image_paths, meta_features, pawpularity=None, transform=None):
        self.image_paths = image_paths
        self.pawpularity = pawpularity
        self.transform = transform
        self.meta_features = meta_features
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image = cv2.imread(self.image_paths[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform is not None:
            image = self.transform(image=image)['image']
        metadata = self.meta_features[index]
        if self.pawpularity is not None:
            label = self.pawpularity[index]
            return image, metadata, label
        else: 
            return image, metadata

In [ ]:
train_dataset = PetDataset(
    image_paths = df_train['image_path'],
    meta_features = df_train[features].values,
    pawpularity = df_train[target],
    transform=get_train_transforms()
)


In [ ]:
def show_image(train_dataset=train_dataset, inline=4):
    plt.figure(figsize=(20,10))
    for i in range(inline):
        rand = random.randint(0, len(train_dataset))
        image, dense, label = train_dataset[rand]
        plt.subplot(1, inline, i%inline +1)
        plt.axis('off')
        plt.imshow(image.permute(2, 1, 0))
        plt.title(f'Pawpularity: {label}')

In [ ]:
for i in range(3):
    show_image(inline=4)

In [ ]:
def calc_rmse_score(output, target):
    y_pred = torch.sigmoid(output).cpu().detach().numpy()*100
    y_label = target.cpu()*100
    return mean_squared_error(y_label, y_pred, squared=False)

In [ ]:
def get_scheduler(optimizer, scheduler_params=params):
    scheduler = None
    if scheduler_params['scheduler_name'] == 'CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(
            optimizer,
            T_0 = scheduler_params['T_0'],
            eta_min = scheduler_params['min_lr']
        )
    return scheduler

In [ ]:
class PetNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'], inp_channels=params['inp_channels'], pretrained=params['pretrained'], num_dense=len(params['dense_features']), dropout=params['dropout']):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, 128)
        self.fc = nn.Sequential(
            nn.Linear(128 + num_dense, 64),
            nn.ReLU(),
            nn.Linear(64, out_features)
        )
        self.dropout = nn.Dropout(dropout)
    def forward(self, image, dense):
        embeddings = self.model(image)
        x = self.dropout(embeddings)
        x = torch.cat([x, dense], dim=1)
        output = self.fc(x)
        return output

In [ ]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"],
                    float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )


In [ ]:
train_val_dataset = PetDataset(
    image_paths = df_train['image_path'].values,
    meta_features = df_train[features].values,
    pawpularity = df_train[target].values/100,
    transform=get_train_transforms()
)

train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_set, val_set = torch.utils.data.random_split(train_val_dataset, [train_size, val_size])

train_loader = DataLoader(
    train_set,
    batch_size=params['batch_size'],
    shuffle=True,
    num_workers=params['num_workers'],
    pin_memory=True
)

val_loader = DataLoader(
    val_set,
    batch_size=params['batch_size'],
    shuffle=False,
    num_workers=params['num_workers'],
    pin_memory=True
)


In [ ]:
model = PetNet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=params['lr'],
    weight_decay=params['weight_decay'],
    amsgrad=False
)
scheduler = get_scheduler(optimizer)

In [ ]:
def train_fn(train_loader, model, criterion, optimizer, epoch, params, scheduler=None):
    model.train()
    metric_monitor = MetricMonitor()
    stream = tqdm(train_loader)
    for i, (images, dense, target) in enumerate(stream, start=1):
        images = images.to(params['device'], non_blocking=True)
        dense = dense.to(params['device'], non_blocking=True)        
        target = target.to(params['device'], non_blocking=True).float().view(-1,1)
        
        output = model(images, dense)
        loss = criterion(output, target)
        rmse_score = calc_rmse_score(output, target)
        metric_monitor.update('Loss', loss.item())
        metric_monitor.update('RMSE', rmse_score)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        stream.set_description(f"Epoch: {epoch:02}. Train. {metric_monitor}")
    if scheduler is not None:
            scheduler.step()

In [ ]:
def validate_fn(val_loader, model, criterion, epoch, param):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    final_targets = []
    final_outputs = []
    with torch.no_grad():
        for i, (images, dense, target) in enumerate(stream, start=1):
            images = images.to(params['device'], non_blocking=True)
            dense = dense.to(params['device'], non_blocking=True)        
            target = target.to(params['device'], non_blocking=True).float().view(-1,1)
            output = model(images, dense)
            loss = criterion(output, target)
            rmse_score = calc_rmse_score(output, target)

            metric_monitor.update('Loss', loss.item())
            metric_monitor.update('RMSE', rmse_score)
            stream.set_description(f"Epoch: {epoch:02}. Train. {metric_monitor}")
            
            targets = (target.detach().cpu().numpy()*100).tolist()
            outputs = (torch.sigmoid(output).detach().cpu().numpy()*100).tolist()

            final_targets.extend(targets)
            final_outputs.extend(outputs)
    return final_outputs, final_targets


In [ ]:
best_rmse = np.inf
best_epoch = np.inf
best_model_name = None
for epoch in range(1, params['epochs']+1):
    train_fn(train_loader, model, criterion, optimizer, epoch, params, scheduler)
    predictions, valid_targets = validate_fn(val_loader, model, criterion, epoch, params)
    rmse = round(mean_squared_error(valid_targets, predictions, squared=False), 3)
    if rmse < best_rmse:
        best_rmse = rmse
        best_epoch = epoch
        if best_model_name is not None:
            os.remove(best_model_name)
        torch.save(model.state_dict(),
                   f"{params['model']}_{epoch}_epoch_{rmse}_rmse.pth")
        best_model_name = f"{params['model']}_{epoch}_epoch_{rmse}_rmse.pth"

In [ ]:
model_dir = './vit_large_patch32_384_2_epoch_24.96_rmse.pth'
model = PetNet().to(device)

model.load_state_dict(torch.load(model_dir))

In [ ]:
test_dataset = PetDataset(
    image_paths = df_test['image_path'].values,
    meta_features = df_test[features].values,
    transform=get_train_transforms()
)

test_loader = DataLoader(
    test_dataset,
    batch_size=params['batch_size'],
    shuffle=False,
    num_workers=params['num_workers'],
    pin_memory=True
)

In [ ]:
def evaluation(test_loader, model, params):
    model.eval()
    final_targets = []
    final_outputs = []

    for i, (images, dense) in enumerate(test_loader):
        images = images.to(params['device'], non_blocking=True)
        dense = dense.to(params['device'], non_blocking=True)
        output = model(images, dense)
        outputs = (torch.sigmoid(output).detach().cpu().numpy()*100).tolist()
        final_outputs.extend(outputs)
    return final_outputs

In [ ]:
predictions = evaluation(test_loader, model, params)
df_test['Pawpularity'] = list(map(lambda x: np.round(x, 2).item(), predictions))

In [ ]:
df_test[['Id','Pawpularity']].to_csv('submission.csv', index=False)